# Riak

In [1]:
import riak
from pprintpp import pprint as pp
import json
import pandas as pd

import uuid
import hashlib

## Lectura de información en Pandas

Partimos del dataset normalizado y lo desnormalizamos para guardarlo todo junto en RIAK

In [2]:
df_mov = pd.read_excel("../../data/black.xlsx", sheetname= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheetname= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
id_miembro            7624 non-null int64
fecha                 7624 non-null datetime64[ns]
minuto                7624 non-null int64
hora                  7624 non-null int64
importe               7624 non-null float64
comercio              6864 non-null object
actividad_completa    7622 non-null object
actividad             7622 non-null object
nombre                7624 non-null object
funcion               7624 non-null object
organizacion          6056 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


In [3]:
df.head(5)

,id_miembro,fecha,minuto,hora,importe,comercio,actividad_completa,actividad,nombre,funcion,organizacion
0,40,2005-03-28,11,16,73.00,LA VACA ARGENTINA,RESTAURANTES RESTO,RESTAURANTE,José María Arteta Vico,concejal,PSOE
1,40,2004-05-31,59,16,10.10,CAFE DE ORIENTE,RESTAURANTES DE 4 Y 5 TENEDORES,RESTAURANTE,José María Arteta Vico,concejal,PSOE
2,40,2007-09-05,43,22,76.29,RESTAURANTE LA VACA ARGEN,RESTAURANTES RESTO,RESTAURANTE,José María Arteta Vico,concejal,PSOE
3,40,2006-12-14,13,0,78.50,EL CAPRICHO,RESTAURANTES DE 4 Y 5 TENEDORES,RESTAURANTE,José María Arteta Vico,concejal,PSOE
4,40,2008-12-12,38,20,59.90,EL CORTE INGLES,EL CORTE INGLES,COMPRA BIENES,José María Arteta Vico,concejal,PSOE


## Riak ...

In [4]:
# connect to database
cliente = riak.RiakClient()
cliente.ping()

True

Utilizamos un bucket para guardar los movimientos en formato JSON y otro para guardar el importe acumulado por miembro.

En el bucket de movimientos utilizamos una clave autogenerada (no vamos a poder localizar los movimientos por clave) y en el bucket con el importe acumulado la clave es el nombre de la persona en formato HASH, para no tener problemas ni con los espacios del nombre ni con los acentos

Se van a crear los siguientes buckets:

|bucket|Clave|Contenido|
|-|-|-|
|movimientos|Marca temporal|Todos los datos del dataset en formnato JSON y desnormalizados|
|acum_importes|Código HASH de la persona que realiza el movimiento|Mapa con el nombre de la persona y el importe acumulado|

El bucket de tweets tendrá los siguientes índices:

|Nombre del índice|Contenido|¿Que busquedas permite hacer?|
|-|-|-|
|idx_miembro_bin|Nombre de la persona|Localizar movimientos de una persona concreta|

In [5]:
BUCKET_MOVIMIENTOS = 'movimientos'
BUCKET_ACUM_IMPORTES = 'acum_importes'

bucket_mov = cliente.bucket(BUCKET_MOVIMIENTOS)
bucket_acum_importes = cliente.bucket_type('maps').bucket(BUCKET_ACUM_IMPORTES)

Función para eliminar los datos de un BUCKET de Riak

In [6]:
def drop_keys(bucket):
    for keys in bucket.stream_keys():
        for key in keys:
            bucket.delete(key)

In [21]:
# Borramos los datos ..
drop_keys(bucket_mov)
drop_keys(bucket_acum_importes)

## Inserción de información en Riak

Para generar los datos en formato JSON partimos del DataFrame de Pandas y lo exportamos a JSON (en formato String) para luego cargarlo en un diccionario de Python

Observa que cuando se genera el dato en formato JSON, los campos fechas se almacenan como un [TIMESTAMP de unix](http://www.unixtimestamp.com), por lo que habría que volver a convertirlo a fecha según el caso

In [8]:
json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

In [9]:
print(len(json_list))

7624


In [10]:
pp(json_list[0])

{
    u'actividad': u'RESTAURANTE',
    u'actividad_completa': u'RESTAURANTES RESTO',
    u'comercio': u'LA VACA ARGENTINA',
    u'fecha': 1111968000000,
    u'funcion': u'concejal',
    u'hora': 16,
    u'id_miembro': 40,
    u'importe': 73.0,
    u'minuto': 11,
    u'nombre': u'José María Arteta Vico',
    u'organizacion': u'PSOE',
}


Vamos a utilizar el nombre de la persona como clave, por lo que necesitamos convertirla previamente a un código que no lleve ni espacios ni acentos.

Este es precisamente el objetivo de la siguiente función ...

In [11]:
def hash_string(s):
    s_utf8 = s.encode('utf-8')
    return hashlib.md5(s_utf8).hexdigest()

Recorremos el dataset de movimientos, almacenando la información tanto en el bucket de movimientos como en el bucket que acumula los movimientos por cliente.

En este último utilizamos una funcionalidad que nos da Riak para guardar la información tipificada. Utilizamos un mapa que tiene dos elementos: Un registro donde guardamos el nombre del cliente y un contador donde guardamos el importe (sin decimales)

In [12]:
# Bucle de carga de datos ...
for movimiento_json in json_list:
    key = uuid.uuid1().hex
    hash_nombre =  hash_string(movimiento_json["nombre"])
    
    movimiento = bucket_mov.new(key, movimiento_json)
    movimiento.add_index('idx_miembro_bin', hash_nombre)
    movimiento.store()
    
    map_nombre = bucket_acum_importes.new(hash_nombre)
    map_nombre.registers['nombre'].assign(movimiento_json["nombre"].encode('utf-8'))
    map_nombre.counters['importe'].increment(int(round(movimiento_json["importe"] * 100)))
    map_nombre.store()
    

# Lectura de información en Riak

En la siguiente función partimos del objeto JSON que nos devuelve RIAK y generamos un DataFrame Pandas.

En el caso de la fecha, lo convertimos desde un UNIX Timestamp al formato correcto para Pandas

In [13]:
def json_to_pandas(rows):
    # Convertimos el objeto JSON en un objeto pandas 
    df = pd.read_json(json.dumps(rows))
    
    if 'fecha' in df.columns:
        # Las fechas están en formato UNIX TIMESTAMP. Las volvemos a convertir a formato Date...
        df = df.assign(fecha = pd.to_datetime(df.fecha, unit = 'ms'))

    return df

### Lectura del bucket de movimientos

Cargamos todos los movimientos de la base de datos y lo guardamos en un objeto Pandas, ya que hay ciertas preguntas que no pueden resolverse directamente por esta base de datos

Recuerda que RIAK permite obtener información de un clave, pero no le es posible devolver la información ordenada ...

In [14]:
rows = []
for keys in bucket_mov.stream_keys():
    for key in keys:
        # print('Key %s' % key )
        rows.append(bucket_mov.get(key).data)
        
# Convertimos el objeto json en un objeto pandas 
df_movimientos = json_to_pandas(rows)

In [15]:
df_movimientos.head()

,actividad,actividad_completa,comercio,fecha,funcion,hora,id_miembro,importe,minuto,nombre,organizacion
0,SUPERMERCADO,ALCAMPO,ALCAMPO-PIO XII,2007-04-25,concejal,14,4,36.43,5,Antonio Cámara Eguinoa,Partido Popular
1,COCHE,AUTOPISTAS (TERMINALES),AUTOPISTA MADRID SUR R4,2005-10-17,concejal,15,4,1.64,45,Antonio Cámara Eguinoa,Partido Popular
2,RESTAURANTE,RESTAURANTES RESTO,RESTAURANTE SANT JAMES,2008-04-22,concejal,23,4,63.00,13,Antonio Cámara Eguinoa,Partido Popular
3,RESTAURANTE,COMIDA RAPIDA,EL SABROSO,2009-04-14,concejal,13,4,4.00,46,Antonio Cámara Eguinoa,Partido Popular
4,COCHE,ACESA,ACESA,2008-08-03,concejal,15,4,19.85,44,Antonio Cámara Eguinoa,Partido Popular


### Los 10 movimientos mas caros

Esta consulta no la podemos contestar directamente con la Base de Datos por lo que nos apoyamos en un proceso en el cliente

In [16]:
df_movimientos.sort_values('importe', ascending=False)[['nombre', 'fecha', 'actividad_completa', 'importe']].head(10)

,nombre,fecha,actividad_completa,importe
3166,Ildefonso José Sánchez Barcoj,2006-02-14,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),11000.00
5559,Ildefonso José Sánchez Barcoj,2008-11-19,EL CORTE INGLES,6593.20
5040,Matías Amat Roca,2010-11-30,AGENCIAS DE VIAJES,6519.12
4337,María Carmen Cafranga Cavestany,2011-02-14,V.DIST.VIAJES Y TRANSPORTE DE VIAJEROS,5500.00
2350,Ildefonso José Sánchez Barcoj,2004-01-08,AGENCIAS DE VIAJES,5283.33
1323,Ricardo Morado Iglesias,2007-02-01,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),5000.00
22,Estanislao Rodríguez-Ponga Salamanca,2009-12-25,EL CORTE INGLES,5000.00
227,Ramón Martínez Vilches,2010-07-07,AGENCIAS DE VIAJES,4955.00
3300,Ricardo Morado Iglesias,2007-06-25,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),4500.00
3956,Ildefonso José Sánchez Barcoj,2010-03-25,EL CORTE INGLES,4320.50


### Lectura a través de un índice

In [17]:
rows = []
keys = bucket_mov.stream_index("idx_miembro_bin", hash_string(u"Javier de Miguel Sánchez"))
for keys in keys.results:
    for movimiento_key in keys:
        rows.append(bucket_mov.get(movimiento_key).data)
    
df = json_to_pandas(rows)

### Los movimientos de una persona concreta

Para obtener los movimientos de una persona en concreto si podemos utilizar el índice que habíamos creado Ad-hoc, aunque la ordenación se realiza en Pandas

In [18]:
df.sort_values('importe', ascending=False)[['nombre', 'fecha', 'actividad_completa', 'importe']].head(10)

,nombre,fecha,actividad_completa,importe
67,Javier de Miguel Sánchez,2010-01-12,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",383.00
124,Javier de Miguel Sánchez,2009-05-30,CONFECCION TEXTIL EN GENERAL,350.00
113,Javier de Miguel Sánchez,2010-01-12,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",345.00
92,Javier de Miguel Sánchez,2009-03-27,HIPERCOR SUPERMERCADOS EL CORTE INGLES,268.50
8,Javier de Miguel Sánchez,2010-01-19,"FERRETERIA,BRICOLAJE,MENAJE DEL HOGAR",260.72
39,Javier de Miguel Sánchez,2004-04-21,JOYERIAS Y RELOJERIAS,240.00
151,Javier de Miguel Sánchez,2009-05-29,HIPERCOR SUPERMERCADOS EL CORTE INGLES,236.05
17,Javier de Miguel Sánchez,2008-12-29,HIPERCOR SUPERMERCADOS EL CORTE INGLES,236.00
115,Javier de Miguel Sánchez,2006-08-30,EL CORTE INGLES,233.39
95,Javier de Miguel Sánchez,2004-07-14,RESTAURANTES RESTO,213.25


### Lectura de una información agregada

La información agregada está en un mapa, por lo que tenemos que procesarla y convertirla a formato JSON.

In [19]:
rows = []
for keys in bucket_acum_importes.stream_keys():
    #print(keys)
    for key in keys:
        map_nombre = bucket_acum_importes.get(key)
        rows.append({'nombre' : map_nombre.registers['nombre'].value,
                     "importe" : float(map_nombre.counters['importe'].value) / 100})
        
# Convertimos el objeto json en un objeto pandas 
df = json_to_pandas(rows)

### Las 10 personas que mas han gastado

Fácil si disponemos de un agregado. Utilizamos Pandas para ordenar la información ..,

In [20]:
df.sort_values('importe', ascending=False).head(10)

,importe,nombre
16,64590.50,Ildefonso José Sánchez Barcoj
75,47292.08,Ricardo Morado Iglesias
10,41207.26,José Antonio Moral Santín
39,39946.33,Juan Manuel Astorqui Portera
82,39820.26,Matías Amat Roca
23,39473.01,Carlos María Martínez Martínez
4,35475.15,Miguel Blesa de la Parra
38,30612.27,Ramón Ferraz Ricarte
52,26030.22,Maria Mercedes de la Merced Monge
25,25113.06,Mariano Pérez Claver
